In [ ]:
import pandas as pd


In [ ]:
import polars as pl
from pathlib import Path

# Pfad zu den Dateien
path_files = 'SRL Soll 2022-2025'
files = [str(p) for p in Path(path_files).glob("*.csv")]

# Lazy Scans für alle CSV-Dateien
lazy_frames = [
    pl.scan_csv(
        f,
        separator=";",
        has_header=True,
        # === KORREKTUR ===
        # Polars anweisen, das Komma als Dezimaltrennzeichen zu verwenden.
        decimal_comma=True, 
        ignore_errors=True 
    )
    for f in files
]

combined_df = pl.concat(lazy_frames, how="vertical")

df_master_ldf = combined_df.select(
    # Datum und Zeit zu einem einzigen DateTime-Objekt zusammenfügen
    pl.concat_str(
        [pl.col("DATE"), pl.col("TIME")],
        separator=" "
    ).str.to_datetime(format="%d.%m.%Y %H:%M:%S").alias("timestamp"),
    
    # Die Setpoint-Spalte auswählen und umbenennen
    pl.col("GERMANY_aFRR_SETPOINT_[MW]")
)

# Erst jetzt werden die Daten eingelesen und alle Operationen ausgeführt
df_master = df_master_ldf.collect(streaming=True)

# Ergebnis anzeigen
print(df_master)
df = df_master.to_pandas()
df


In [17]:
# include localize europe german timezone


df.index = df.index.tz_localize(
    'Europe/Berlin',
    ambiguous='infer',         # bei Zweifel: vorherige Information nutzen
    nonexistent='shift_forward'  # fehlende Zeiten im Frühjahr eine Stunde vorziehen
)

df


,GERMANY_aFRR_SETPOINT_[MW]
timestamp,
2022-01-01 00:00:00+01:00,-78.723
2022-01-01 00:00:01+01:00,-78.723
2022-01-01 00:00:02+01:00,-43.437
2022-01-01 00:00:03+01:00,-43.437
2022-01-01 00:00:04+01:00,-30.867
...,...
2025-06-29 23:59:55+02:00,0.000
2025-06-29 23:59:56+02:00,0.000
2025-06-29 23:59:57+02:00,0.000


In [18]:
# negative vaues in 'srl negative' pos values in 'srl positive'
df_srl_sollwert = df.copy()
df_srl_sollwert['srl_positive'] = df_srl_sollwert['GERMANY_aFRR_SETPOINT_[MW]'].clip(lower=0)
df_srl_sollwert['srl_negative'] = df_srl_sollwert['GERMANY_aFRR_SETPOINT_[MW]'].clip(upper=0)
df_srl_sollwert.to_pickle('srl_setpoint.pkl')



In [19]:
df_4s_mean = df_srl_sollwert.resample('4S').mean()
df_4s_mean.to_pickle('srl_setpoint_4s.pkl')


C:\Users\AdminMichael\AppData\Local\Temp\32\ipykernel_14552\3747524139.py:1: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_4s_mean = df_srl_sollwert.resample('4S').mean()


In [ ]:
# disable max rows display limit
import pandas as pd
pd.set_option('display.max_rows', 12)  # alle Zeilen anzeigen

df = pd.read_pickle('srl_setpoint_4s.pkl')
print(df)#['2022-06-01':'2022-06-30'])